In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import *
import datetime
import time
from datetime import date, timedelta
from datetime import date
from pyspark.sql.window import Window
import pysftp
import os

In [0]:
file_generated_timestamp      = datetime.datetime.now().strftime('%Y%m%d%H%M')
destination_table             = "data_suppression.perfect_golden_ufs_recipientscores_" + str(file_generated_timestamp)
destination_table_acm         = "data_suppression.perfect_golden_wunderman_recipientscores_" + str(file_generated_timestamp)
export_timestamp              = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
print(destination_table)

data_suppression.perfect_golden_ufs_recipientscores_202006021200

In [0]:
dbfs_mount_path               = "dbfs:/mnt/menaka"
csv_folderpath                = "%s/suppression_model/production/" % dbfs_mount_path
wunderman_export_filename     = "PerfectGolden_Wunderman_RecipientScores_" + str(file_generated_timestamp) 

print ('DBFS File path : ' , csv_folderpath)
print(wunderman_export_filename)

DBFS File path : dbfs:/mnt/menaka/suppression_model/production/
PerfectGolden_Wunderman_RecipientScores_202005030951

In [0]:
ufs_scores       = spark.sql("""show tables in data_suppression""")
ufs_scores       = ufs_scores.where(col("tableName").rlike("perfect_golden_ufs_recipientscores"))
ufs_scores       = ufs_scores.withColumn("date", substring(col("tableName"),36,100))

window           = Window.partitionBy('database').orderBy(col('date').desc())
ufs_scores       = ufs_scores.withColumn("rank",rank().over(window))
ufs_scores       = ufs_scores.where(col("rank") <= 3)
display(ufs_scores)

In [0]:
previous_score_table              = "data_suppression." + ufs_scores.where(col("rank") == 2).select("tableName").rdd.max().tableName
previous_score                    = spark.table(previous_score_table)
#previous_score                    = previous_score.where(col("cp_countryCode") == "FR")
#display(previous_score.groupby("cp_countryCode","Score").agg(countDistinct(col("cp_ohubId")).alias("NoofRcp")))

previous_score                    = previous_score.select("cp_ohubId","Score")
previous_score                    = previous_score.withColumnRenamed("cp_ohubId","ohubId")
previous_score                    = previous_score.withColumnRenamed("Score","OldScore")
#print(previous_score_table)

In [0]:
current_score_table              = "data_suppression." + ufs_scores.where(col("rank") == 1).select("tableName").rdd.max().tableName
current_score                    = spark.table(current_score_table)
#current_score                    = current_score.where(col("cp_countryCode") == "FR")
#display(current_score.groupby("cp_countryCode","Score").agg(countDistinct(col("cp_ohubId")).alias("NoofRcp")))
#print(current_score_table)

In [0]:
forced_scores = current_score.join(previous_score, previous_score.ohubId == current_score.cp_ohubId, how = "left")

forced_scores = forced_scores.withColumn("Score",
         when( (col("Score") != 6)     &  (col("OldScore") == 6)  & (col("Opens_Less_6_Months") == 0)   ,6)                                   
        .when( (col("OldScore") == 6)  &  (col("Score") == 6) ,6)
        .when( (col("Score") == 7)     &  (col("OldScore") == 3)  & (col("Opens_Less_6_Months") == 0), 3)
        .when( (col("Score") == 7)     &  (col("OldScore") == 4)  & (col("Opens_Less_6_Months") == 0), 4)    
        .when( (col("Score") == 7)     &  (col("OldScore") == 5)  & (col("Opens_Less_6_Months") == 0), 5)  
#         .when( (col("Score") == 7)     &  (col("Opens_Less_6_Months") > 0)    , 11)
#         .when( (col("Score") == 7)     &  (col("Opens_Less_6_Months") == 0)   , 12)                                 
                                         .otherwise(col("Score"))
                                        )
forced_scores = forced_scores.drop("OldScore","ohubId")
#display(forced_scores.groupby("cp_countryCode","NewScore").agg(countDistinct(col("cp_ohubId")).alias("NoofRcp")))

In [0]:
forced_scores.write.mode("overwrite").saveAsTable(destination_table)

In [0]:
wunderman_suppression_scores_final = forced_scores.withColumn("Score", when(col("Score").isin(1,2,7,10,-3), 1) #Keep sending all emails
                                                              .when(col("Score") == 6,3) # Suppress operator 
                                                              .when( col("Score").isin(3,-1,0), 6) # Re-engagement campaigns  
                                                              .when(col("Score").isin(4), 7) # Re-engagement campaigns
                                                              .when(col("Score").isin(5), 8) # Re-engagement campaigns
                                                             
                                                              .otherwise(col("Score")))

wunderman_suppression_scores_final = wunderman_suppression_scores_final.withColumn("Email_Action", 
 when(col("Score") == 1, 'Keep_Sending_Emails')
.when(col("Score") == 3, 'Suppress_Operator')      
.when(col("Score") == 6, 'Start_Reactivation _Campaign_1')         
.when(col("Score") == 7, 'Start_Reactivation _Campaign_2') 
.when(col("Score") == 8, 'Start_Last_Off_Reactivation')                                                                                   
)

wunderman_suppression_scores_final.write.mode("overwrite").saveAsTable(destination_table_acm)

In [0]:
%run "/Shared/dataPipelines/OperatorLifeCycle/S01_Libraries_Functions"

In [0]:
wunderman_data_format = wunderman_suppression_scores_final.withColumn("ScoreType", lit("EMAIL_SUPPRESSION"))
wunderman_data_format = wunderman_data_format.withColumn("Calculationdate", lit(export_timestamp))

wunderman_data_format = wunderman_data_format.select(col("cp_ohubId").alias("RecipientPartyID"), "ScoreType","Calculationdate","Score").distinct()

wunderman_data_format.repartition(1).write.option("sep",",").csv(csv_folderpath + wunderman_export_filename +".csv" , mode="overwrite" , header=True)
copyMerge(csv_folderpath + wunderman_export_filename +".csv", csv_folderpath + wunderman_export_filename + "_merged.csv", debug=True, overwrite=True, deleteSource=True)

Appending file dbfs:/mnt/menaka/suppression_model/production/PerfectGolden_Wunderman_RecipientScores_202004261925.csv/part-00000-tid-6209942868833730113-4dd521f7-d3f4-49ff-b3bd-8be030fc204c-21775-1-c000.csv into dbfs:/mnt/menaka/suppression_model/production/PerfectGolden_Wunderman_RecipientScores_202004261925_merged.csv
Source directory dbfs:/mnt/menaka/suppression_model/production/PerfectGolden_Wunderman_RecipientScores_202004261925.csv removed.

In [0]:
host                     = 'unilever-sftp.neolane.net'
port                     = 22
user                     = dbutils.secrets.get(scope = 'unileversftp', key = 'username')
password                 = dbutils.secrets.get(scope = 'unileversftp', key = 'password')



#local                    = '/dbfs:/mnt/menaka/suppression_model/production/' 
local_filename           = '/dbfs/mnt/menaka/suppression_model/production/' + str(wunderman_export_filename) +"_merged.csv"
remote                   = '/incoming/import/RecipientScores_' + str(file_generated_timestamp) + ".csv" 
cnopts                   = pysftp.CnOpts()
cnopts.hostkeys          = None

print(local_filename)
print(remote)

/databricks/python/lib/python3.7/site-packages/pysftp/__init__.py:61: UserWarning: Failed to load HostKeys from /root/.ssh/known_hosts. You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
 warnings.warn(wmsg, UserWarning)
/dbfs/mnt/menaka/suppression_model/production/PerfectGolden_Wunderman_RecipientScores_202004261925_merged.csv
/incoming/import/RecipientScores_202004261925.csv

In [0]:
time.sleep(5)

In [0]:
while True:
  try:
    with pysftp.Connection(host=host, username=user, password=password, cnopts=cnopts) as sftp:
      print ("Connection succesfully stablished ... ")
      sftp.put(local_filename,remote)  
      sftp.close()
      print ("File Uploaded succesfully")
      break     
  except Exception as e:
    print('Upload Exception: ' + str(e))

Connection succesfully stablished ... 
File Uploaded succesfully

In [0]:
table_name =  "data_suppression." + ufs_scores.where(col("rank") == 1).select("tableName").rdd.max().tableName

In [0]:
syntax = "DROP TABLE IF EXISTS " + str(table_name)
spark.sql(syntax)